# EDA descriptivo de variables clave por país

**Objetivo:** Realizo un análisis descriptivo de variables clínicas clave (SLEDAI, C3, C4, anti-dsDNA, CRP, Vitamina D y TyG) por país (México, Brasil, España). Para cada variable estimo:
- número válidos y % nulos,
- rango (mín, máx), media, mediana y desviación estándar.
- número de outliers por método IQR.

Al final genero una **tabla resumen por país** y un **texto de conclusión** breve para la Metodología.


In [1]:
import os
import re
import numpy as np
import pandas as pd

pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", 50)
pd.set_option("display.float_format", lambda x: f"{x:.3f}")


## Carga de datos integrados

Trabajo directamente con el archivo procesado `dataset_ready.xlsx`  
(ruta: `TFM MÁSTER BIOINFORMÁTICA/outputs/dataset_ready.xlsx`).

Este fichero ya contiene la información integrada y depurada, por lo que no necesito unir varias hojas.


In [4]:
import pandas as pd

PATH_XLS = "outputs/dataset_ready.xlsx"
df = pd.read_excel(PATH_XLS)
print("Dimensiones:", df.shape)
df.head()



Dimensiones: (290, 58)


,Race,Gender,Age (years),Marital status,Education level,Smoking habits,Time of disease (years),HCQ use (mg/day),SLICC,SLEDAI,Weight (kg),Height (m),BMI (kg/m2),Waist Circ (cm),Systolic Blood Pressure (mm/Hg),Diastolic Blood Pressure (mm/Hg),Glucose (mg/dL),Total cholesterol (mg/dL),LDL (mg/dL),HDL (mg/dL),Triglycerides (mg/dL),C-reactive protein,Albumin (g/dL),GOT_AST (U/L),GPT_ALT (U/L),...,Monocytes,Platelets,Hemoglobin,Hematocrit,VCM,CHCM,RDW,VSG (mm),C3 complement,C4 complement,Anti-dsDNA,TyG,Energy intake (kcal/day),Carbohydrate intake (g/day),Carbohydrate intake (%TEI),Protein intake (g/day),Protein intake (%TEI),Lipid intake (g/day),Lipid (%TEI),METs-min/week,IPAQ,FACIT Fatigue Scale,PCS12 (HRQoL),MCS12 (HRQoL),Country
0,Mexican-Mestizo,Female,29,married,Incomplete academic degree,No,4.000,NaN,0.000,4.000,95.900,1.610,36.900,111.100,143.000,66,75.000,175.000,73.800,40.000,306.000,8.510,4.000,NaN,NaN,...,8.220,261.000,13.140,39.900,39.900,89.100,NaN,NaN,118.000,23.000,P,NaN,1356.150,217.185,NaN,87.005,NaN,64.620,NaN,NaN,Sedentar,NaN,NaN,NaN,Mexico
1,Mexican-Mestizo,Female,24,married,Incomplete academic degree,No,4.000,NaN,0.000,0.000,41.600,1.540,30.175,65.500,102.000,63,79.000,112.000,59.350,23.820,106.680,15.920,3.440,NaN,NaN,...,3.670,344.500,13.140,28.490,28.490,90.690,NaN,NaN,106.000,17.525,P,NaN,1356.150,217.185,NaN,87.005,NaN,64.620,NaN,NaN,Sedentar,NaN,NaN,NaN,Mexico
2,Mexican-Mestizo,Female,66,widow,Incomplete academic degree,No,16.000,NaN,4.000,0.000,67.900,1.490,30.175,93.700,137.000,77,124.980,198.990,126.340,51.130,119.040,3.590,3.870,NaN,NaN,...,6.160,231.800,13.280,42.610,42.610,98.750,NaN,NaN,143.500,16.590,N,NaN,1356.150,217.185,NaN,41.390,NaN,64.620,NaN,NaN,Sedentar,NaN,NaN,NaN,Mexico
3,Mexican-Mestizo,Female,29,single,Incomplete academic degree,yes,1.000,NaN,1.000,4.000,72.700,1.640,27.000,93.500,122.000,78,86.930,113.450,54.070,38.430,137.160,3.020,4.000,NaN,NaN,...,8.220,232.000,13.140,39.715,39.715,89.590,NaN,NaN,106.000,17.525,P,NaN,1356.150,217.185,NaN,87.005,NaN,64.620,NaN,NaN,Activo,NaN,NaN,NaN,Mexico
4,Mexican-Mestizo,Female,40,single,Incomplete academic degree,No,13.000,NaN,1.000,0.000,60.100,1.620,30.175,79.800,114.000,77,97.000,157.000,78.570,66.670,66.000,3.190,3.720,NaN,NaN,...,8.220,210.000,13.140,47.600,47.600,91.200,NaN,NaN,106.000,17.525,N,NaN,1356.150,217.185,NaN,87.005,NaN,52.700,NaN,NaN,Sedentar,NaN,NaN,NaN,Mexico


## Inspección de columnas



In [7]:
cols = df.columns.tolist()
print("Número de columnas:", len(cols))
for i, c in enumerate(cols, start=1):
    print(f"{i:2d}. {c}")


Número de columnas: 58
 1. Race
 2. Gender
 3. Age (years)
 4. Marital status
 5. Education level
 6. Smoking habits
 7. Time of disease (years)
 8. HCQ use (mg/day)
 9. SLICC
10. SLEDAI
11. Weight (kg)
12. Height (m)
13. BMI (kg/m2)
14. Waist Circ (cm)
15. Systolic Blood Pressure (mm/Hg)
16. Diastolic Blood Pressure (mm/Hg)
17. Glucose (mg/dL)
18. Total cholesterol (mg/dL)
19. LDL (mg/dL)
20. HDL (mg/dL)
21. Triglycerides (mg/dL)
22. C-reactive protein
23. Albumin (g/dL)
24. GOT_AST (U/L)
25. GPT_ALT (U/L)
26. Urea (mg/dL)
27. Creatinine (mg/dL)
28. Folic acid (ng/mL)
29. Vitamin B12 (ng/ml)
30. Vitamin D (ng/mL)
31. Leukocytes
32. Neutrophils
33. Lymphocytes
34. Monocytes
35. Platelets
36. Hemoglobin
37. Hematocrit
38. VCM
39. CHCM
40. RDW
41. VSG (mm)
42. C3 complement
43. C4 complement
44. Anti-dsDNA
45. TyG
46. Energy intake (kcal/day)
47. Carbohydrate intake (g/day)
48. Carbohydrate intake (%TEI)
49. Protein intake (g/day)
50. Protein intake (%TEI)
51. Lipid intake (g/day)
52. Li

## Selección y jerarquía de variables (alineado con recomendaciones de tutora)

Defino la jerarquía de variables que usaré a partir de la literatura y de las recomendaciones de mi tutora:

- **Variable dependiente principal**
  - **SLEDAI**: índice clínico validado y ampliamente utilizado para actividad de enfermedad en lupus.

- **Variables dependientes secundarias / complementarias (actividad inmunológica e inflamatoria)**
  - **C3 complement**, **C4 complement**, **Anti-dsDNA**, **C-reactive protein**.

- **Variables explicativas / covariables (nutrición y metabolismo)**
  - **Vitamin D (ng/mL)**, **TyG**.

> Nota: Aunque FACIT Fatigue es relevante para calidad de vida, **no la incluiré** como dependiente en.de la tutora.

En el EDA descriptivo reportaré, por **país** (Brazil, Spain, Mexico), para cada variable:  
*n válidos, % nulos, mínimo, máximo, media, mediana, desviación estándar y outliers (IQR)*.  
Este bloque también me permitirá valorar si **SLEDAI** es viable como dependiente principal en términos de cobertura.


In [8]:
# Defino los grupos de variables según la jerarquía acordada
var_dep_principal = ["SLEDAI"]

var_dep_secundarias = [
    "C3 complement",
    "C4 complement",
    "Anti-dsDNA",
    "C-reactive protein",
]

var_explicativas = [
    "Vitamin D (ng/mL)",
    "TyG",
]

# Para el EDA por país necesito también 'Country'
vars_clave = var_dep_principal + var_dep_secundarias + var_explicativas + ["Country"]

# Verificación de presencia en el dataframe
presentes = [v for v in vars_clave if v in df.columns]
faltantes = [v for v in vars_clave if v not in df.columns]

print("Variables encontradas:", presentes)
print("Variables faltantes:", faltantes)

# Subconjunto de trabajo (solo si todo lo esencial está presente)
df_clave = df[presentes].copy()
df_clave.head()


Variables encontradas: ['SLEDAI', 'C3 complement', 'C4 complement', 'Anti-dsDNA', 'C-reactive protein', 'Vitamin D (ng/mL)', 'TyG', 'Country']
Variables faltantes: []


,SLEDAI,C3 complement,C4 complement,Anti-dsDNA,C-reactive protein,Vitamin D (ng/mL),TyG,Country
0,4.000,118.000,23.000,P,8.510,26.910,NaN,Mexico
1,0.000,106.000,17.525,P,15.920,14.160,NaN,Mexico
2,0.000,143.500,16.590,N,3.590,27.270,NaN,Mexico
3,4.000,106.000,17.525,P,3.020,33.270,NaN,Mexico
4,0.000,106.000,17.525,N,3.190,25.130,NaN,Mexico


## Estadísticos descriptivos por país

Calculo, para cada variable y cada país (Brazil, Spain, Mexico):

- `n_total`
- `n_válidos` y `% nulos`
- Para variables numéricas: mínimo, máximo, media, mediana, desviación estándar
- Número de outliers detectados con criterio IQR (1.5×IQR)

**Nota**: La variable *Anti-dsDNA* es categórica (positivo/negativo).  
En este caso reporto únicamente los conteos y porcentajes por categoría en lugar de estadísticos numéricos.


In [9]:
import numpy as np
import pandas as pd

def summarize_numeric(series: pd.Series):
    """Calcula resumen descriptivo con outliers por IQR."""
    s = pd.to_numeric(series, errors="coerce").dropna()
    if s.empty:
        return dict(n_valid=0, min=np.nan, max=np.nan, mean=np.nan,
                    median=np.nan, std=np.nan, n_out=0)
    q1, q3 = s.quantile(0.25), s.quantile(0.75)
    iqr = q3 - q1
    lower, upper = q1 - 1.5*iqr, q3 + 1.5*iqr
    n_out = ((s < lower) | (s > upper)).sum()
    return dict(
        n_valid=int(s.shape[0]),
        min=float(s.min()),
        max=float(s.max()),
        mean=float(s.mean()),
        median=float(s.median()),
        std=float(s.std(ddof=1)),
        n_out=int(n_out),
    )

countries = ["Brazil", "Spain", "Mexico"]
rows = []

for country in countries:
    dfc = df_clave[df_clave["Country"] == country]
    n_total = int(dfc.shape[0])

    for var in var_dep_principal + var_dep_secundarias + var_explicativas:
        if var == "Anti-dsDNA":
            # Tratamiento especial: variable categórica
            counts = dfc[var].value_counts(dropna=False).to_dict()
            n_valid = sum(counts.values()) - (counts.get(np.nan, 0))
            pct_null = (100 * (1 - n_valid/n_total)) if n_total > 0 else np.nan
            rows.append({
                "Country": country,
                "Variable": var,
                "n_total": n_total,
                "n_valid": n_valid,
                "% null": round(pct_null, 2),
                "Counts": counts
            })
        else:
            stats = summarize_numeric(dfc[var])
            n_valid = stats["n_valid"]
            pct_null = (100 * (1 - n_valid/n_total)) if n_total > 0 else np.nan
            rows.append({
                "Country": country,
                "Variable": var,
                "n_total": n_total,
                "n_valid": n_valid,
                "% null": round(pct_null, 2),
                "min": stats["min"],
                "max": stats["max"],
                "mean": stats["mean"],
                "median": stats["median"],
                "std": stats["std"],
                "outliers_IQR": stats["n_out"]
            })

summary_df = pd.DataFrame(rows)
summary_df


,Country,Variable,n_total,n_valid,% null,min,max,mean,median,std,outliers_IQR,Counts
0,Brazil,SLEDAI,143,143,0.000,0.000,8.000,0.839,0.000,1.685,17.000,NaN
1,Brazil,C3 complement,143,143,0.000,45.000,189.000,105.441,106.000,26.122,2.000,NaN
2,Brazil,C4 complement,143,143,0.000,2.000,43.600,20.207,18.950,7.686,0.000,NaN
3,Brazil,Anti-dsDNA,143,143,0.000,NaN,NaN,NaN,NaN,NaN,NaN,"{'N': 113, 'P': 30}"
4,Brazil,C-reactive protein,143,143,0.000,0.300,54.400,5.359,2.300,9.047,13.000,NaN
5,Brazil,Vitamin D (ng/mL),143,143,0.000,11.500,58.800,32.101,31.000,9.257,0.000,NaN
6,Brazil,TyG,143,143,0.000,7.222,11.459,8.351,8.246,0.606,4.000,NaN
7,Spain,SLEDAI,77,0,100.000,NaN,NaN,NaN,NaN,NaN,0.000,NaN
8,Spain,C3 complement,77,77,0.000,20.800,142.000,96.810,99.000,23.485,2.000,NaN
9,Spain,C4 complement,77,77,0.000,3.200,75.000,19.445,17.300,10.575,2.000,NaN


## Exportación de tabla resumen

Guardo la tabla `summary_df` con todas las métricas en CSV y Excel.  
Esto permite documentar el análisis y reutilizar los resultados en el TFM o en futuras revisiones.


In [10]:
out_csv  = "outputs/eda_variables_clave_por_pais.csv"
out_xlsx = "outputs/eda_variables_clave_por_pais.xlsx"

summary_df.to_csv(out_csv, index=False)
summary_df.to_excel(out_xlsx, index=False)

print("Archivos guardados en:", out_csv, "y", out_xlsx)


Archivos guardados en: outputs/eda_variables_clave_por_pais.csv y outputs/eda_variables_clave_por_pais.xlsx


## Conclusión (borrador para Metodología)

El análisis descriptivo por país mostró diferencias claras en la cobertura de las variables clínicas.  
La variable **SLEDAI** se encuentra completa en las cohortes de Brazil (n=143) y Mexico (n=70), pero ausente en Spain (n=77).  
Por tanto, es viable considerarla como **variable dependiente principal** para el estudio, con la limitación de que el análisis se centrará en los pacientes de Brazil y Mexico.

Los biomarcadores inmunológicos y de inflamación presentan buena cobertura global:  
- **C3 y C4 complement**: disponibles en los tres países con rangos fisiológicamente plausibles.  
- **Anti-dsDNA**: registrado como categórico (P/N) en Brazil y Mexico, mientras que en Spain se expresa como valores cuantitativos, lo que requerirá armonización.  
- **CRP**: disponible en todas las cohortes, con presencia de outliers esperables en procesos inflamatorios.

Entre las covariables nutricionales y metabólicas:  
- **Vitamin D** muestra buena cobertura en todos los países, aunque con valores notablemente más elevados en Spain.  
- **TyG** presenta buena cobertura en Brazil y Spain, pero ausencia total de datos en Mexico.

En conjunto, estos resultados apoyan el uso de **SLEDAI** como variable dependiente principal y de **C3, C4, Anti-dsDNA y CRP** como complementarias.  
Las covariables **Vitamin D** y **TyG** aportan información adicional sobre nutrición y metabolismo, aunque la falta de datos de TyG en Mexico debe tenerse en cuenta en los análisis posteriores.
